In [43]:
import pandas as pd
from sqlite3 import connect
import math
import matplotlib.pyplot as plt
import numpy as np
pd.set_option('mode.chained_assignment', None) # gets rid of an unhelpful SettingWithCopyWarning later on down the line.
pd.set_option("display.max_colwidth", None)

This results.csv file to be cleaned and analyzed here was compiled by Mikhail Zhilkin and is hosted at https://data.world/cervus/sumo-japan

This results.csv file lists every individual match for every wrestler on every day of every tournament between January 1983 and July 2023. My goal with this analysis project is to take this data and compile a new dataframe, one that focuses on giving a row to each individual sumo wrestler as opposed to each match outcome. This new dataframe will feature each wrestler along with the wrestler's compiled statistics (wins, losses, top rank, favorite techniques, etc.) I will later import a second CSV file (banzuke.csv), which features rows going tournament-to-tournament with information on each wrestler's height, weight, place of origin, etc. With that second spreadsheet, I will also transform that data into a new dataframe that similarly features each wrestler's compiled statistics. Once I have these two new dataframes, I will combine them into a new spreadsheet, analysis.csv, from which I will then conduct further analysis.

In [44]:
results = pd.read_csv("data/results.csv", dtype={'basho': 'string', 'day': 'string', 'rikishi1_id': 'int64', 'rikishi1_rank': 'string', 'rikishi1_shikona': 'string', 'rikishi1_result': 'string', 'rikishi1_win': 'int64', 'kimarite': 'string', 'rikishi2_id': 'int64', 'rikishi2_rank': 'string', 'rikishi2_shikona': 'string', 'rikishi2_result': 'string', 'rikishi2_win': 'string'})
results

,basho,day,rikishi1_id,rikishi1_rank,rikishi1_shikona,rikishi1_result,rikishi1_win,kimarite,rikishi2_id,rikishi2_rank,rikishi2_shikona,rikishi2_result,rikishi2_win
0,1983.01,1,4140,J13w,Chikubayama,0-1 (7-8),0,yorikiri,4306,Ms1e,Ofuji,1-0 (6-1),1
1,1983.01,1,4306,Ms1e,Ofuji,1-0 (6-1),1,yorikiri,4140,J13w,Chikubayama,0-1 (7-8),0
2,1983.01,1,1337,J12w,Tochitsukasa,1-0 (9-6),1,oshidashi,4323,J13e,Shiraiwa,0-1 (3-12),0
3,1983.01,1,4323,J13e,Shiraiwa,0-1 (3-12),0,oshidashi,1337,J12w,Tochitsukasa,1-0 (9-6),1
4,1983.01,1,4097,J12e,Tamakiyama,0-1 (8-7),0,yorikiri,4319,J11w,Harunafuji,1-0 (5-10),1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
227767,2023.07,15,11985,S1w,Daieisho,9-6,0,hikiotoshi,11855,M9e,Takanosho,8-7,1
227768,2023.07,15,12451,S1e,Hoshoryu,12-3,1,uwatenage,12796,M17w,Hakuoho,11-4,0
227769,2023.07,15,12796,M17w,Hakuoho,11-4,0,uwatenage,12451,S1e,Hoshoryu,12-3,1
227770,2023.07,15,12094,K1w,Abi,6-9,1,yorikiri,12231,O1w,Kirishima,6-7-2,0


Sumo wrestlers (rikishi) in these datasets can best be identified by their unique ID numbers. Although each rikishi has his own name (shikona), some wrestlers may reuse the same name, and other wrestlers may change names during their career. Since the ID number is the one identifier that remains stable,
this is the best identifier to which I will anchor my analysis. Here I make a quick list of unique ID numbers for later use.

In [45]:
rikishi_id_list = []

for i in results["rikishi1_id"].unique():
    rikishi_id_list.append(i)

# Next to make a dictionary that holds each wrestler's unique ID number as a key, with values being each name associated with that wrestler.
# The vast majority of ID numbers will only feature one name, but this will make sure to include any additional names as well.

rikishi_id_dict = {}

# Initializes a key for every number results["rikishi1_id"] but still only adds one shikona (wrestler name) as a value per key, overwriting old shikona.
# So while this fills out the dictionary, it remains incomplete.
for i in range(len(results)):
    rikishi_id_dict.update({results["rikishi1_id"][i]: [results["rikishi1_shikona"][i]]})

# This fills out the dictionary with all associated names.
# Easy to test on rikishi_id_dict[1111], which shows both names for this wrestler, "Ama" and "Harumafuji."
# The first entry in every names list is the *most recent* name the wrestler used. So rikishi_id_dict[1111][0] shows "Harumafuji."
# This is useful to know, because when referring to a wrestler in the present day, one generally refers to them by their most current name.
# So when we reference this dictionary to indicate a wrestler's name, we will go by their most recent name, even if they have used others in the past.
for index, row in results.iterrows():
    if row["rikishi1_shikona"] not in rikishi_id_dict[row["rikishi1_id"]]:
        rikishi_id_dict[row["rikishi1_id"]].append(row["rikishi1_shikona"])

rikishi_id_dict

{4140: ['Chikubayama'],
 4306: ['Ofuji'],
 1337: ['Tochitsukasa'],
 4323: ['Shiraiwa'],
 4097: ['Tamakiyama'],
 4319: ['Harunafuji'],
 4109: ['Kotogatake'],
 4129: ['Hakuryuyama'],
 4111: ['Zaonishiki'],
 4130: ['Hachiya'],
 1302: ['Takamisugi'],
 4318: ['Tengoyama'],
 1361: ['Daitetsu'],
 4316: ['Kakureizan'],
 1385: ['Tochiakagi'],
 4125: ['Shinko'],
 1347: ['Itai'],
 4311: ['Hakuryu'],
 1332: ['Sakahoko'],
 4322: ['Sanofuji'],
 1380: ['Hidanohana'],
 4128: ['Kotochitose'],
 1350: ['Onokuni'],
 1359: ['Koboyama'],
 4087: ['Wakajishi'],
 4091: ['Washuyama'],
 1342: ['Asahifuji'],
 4103: ['Banryuyama'],
 4122: ['Kurosegawa'],
 4131: ['Tochitsurugi'],
 1348: ['Jingaku'],
 4124: ['Shishiho'],
 1334: ['Wakasegawa'],
 4118: ['Amanoyama'],
 1345: ['Tamaryu'],
 4100: ['Kaneshiro', 'Tochihikari'],
 1352: ['Tagaryu'],
 4132: ['Wakanofuji'],
 4050: ['Takamiyama'],
 4117: ['Misugiiso', 'Azumanada'],
 1378: ['Hoo'],
 4133: ['Takarakuni', 'Saisu'],
 1338: ['Ozutsu'],
 4077: ['Fujizakura'],
 1375: 

Now to creates a dataframe using this dictionary, listing each wrestler's ID with any shikona (name) associated with that wrestler. As mentioned in the previous cell, the first value in the shikona list is the most recent name the wrestler has used, which is why I'm making it the default with the column "shikona," while older names are listed as "alt_shikona."

In [46]:
shikona_df = pd.DataFrame.from_dict([([key] + value) for key, value in rikishi_id_dict.items()])
shikona_df.rename(columns={0: "id", 1: "rikishi", 2: "alt_shikona2", 3: "alt_shikona3"}, inplace=True)
shikona_df.head()

,id,rikishi,alt_shikona2,alt_shikona3
0,4140,Chikubayama,None,None
1,4306,Ofuji,None,None
2,1337,Tochitsukasa,None,None
3,4323,Shiraiwa,None,None
4,4097,Tamakiyama,None,None


There are two lines for each match in the original csv file, one for the winner and one for the loser. So as not to show duplicates of every match, we'll just show the winners with the "sumo_only_wins" list.

In [47]:
sumo_only_wins = results[results.rikishi1_win == int("1")] #could also use results.loc[results.rikishi1_win == 1]
sumo_only_wins

,basho,day,rikishi1_id,rikishi1_rank,rikishi1_shikona,rikishi1_result,rikishi1_win,kimarite,rikishi2_id,rikishi2_rank,rikishi2_shikona,rikishi2_result,rikishi2_win
1,1983.01,1,4306,Ms1e,Ofuji,1-0 (6-1),1,yorikiri,4140,J13w,Chikubayama,0-1 (7-8),0
2,1983.01,1,1337,J12w,Tochitsukasa,1-0 (9-6),1,oshidashi,4323,J13e,Shiraiwa,0-1 (3-12),0
5,1983.01,1,4319,J11w,Harunafuji,1-0 (5-10),1,yorikiri,4097,J12e,Tamakiyama,0-1 (8-7),0
7,1983.01,1,4129,J11e,Hakuryuyama,1-0 (3-12),1,tsukidashi,4109,J10w,Kotogatake,0-1 (7-8),0
9,1983.01,1,4130,J9w,Hachiya,1-0 (8-7),1,hatakikomi,4111,J10e,Zaonishiki,0-1 (4-11),0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
227763,2023.07,15,12270,K1e,Kotonowaka,11-4,1,yorikiri,6594,M15e,Ryuden,10-5,0
227765,2023.07,15,12291,M4e,Asanoyama,8-4-3,1,yorikiri,11980,S2w,Wakamotoharu,9-6,0
227766,2023.07,15,11855,M9e,Takanosho,8-7,1,hikiotoshi,11985,S1w,Daieisho,9-6,0
227768,2023.07,15,12451,S1e,Hoshoryu,12-3,1,uwatenage,12796,M17w,Hakuoho,11-4,0


Same as above, but now we'll just show the losers with the "sumo_only_losses" list.

In [48]:
sumo_only_losses = results[results.rikishi1_win == int("0")]
sumo_only_losses

,basho,day,rikishi1_id,rikishi1_rank,rikishi1_shikona,rikishi1_result,rikishi1_win,kimarite,rikishi2_id,rikishi2_rank,rikishi2_shikona,rikishi2_result,rikishi2_win
0,1983.01,1,4140,J13w,Chikubayama,0-1 (7-8),0,yorikiri,4306,Ms1e,Ofuji,1-0 (6-1),1
3,1983.01,1,4323,J13e,Shiraiwa,0-1 (3-12),0,oshidashi,1337,J12w,Tochitsukasa,1-0 (9-6),1
4,1983.01,1,4097,J12e,Tamakiyama,0-1 (8-7),0,yorikiri,4319,J11w,Harunafuji,1-0 (5-10),1
6,1983.01,1,4109,J10w,Kotogatake,0-1 (7-8),0,tsukidashi,4129,J11e,Hakuryuyama,1-0 (3-12),1
8,1983.01,1,4111,J10e,Zaonishiki,0-1 (4-11),0,hatakikomi,4130,J9w,Hachiya,1-0 (8-7),1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
227762,2023.07,15,6594,M15e,Ryuden,10-5,0,yorikiri,12270,K1e,Kotonowaka,11-4,1
227764,2023.07,15,11980,S2w,Wakamotoharu,9-6,0,yorikiri,12291,M4e,Asanoyama,8-4-3,1
227767,2023.07,15,11985,S1w,Daieisho,9-6,0,hikiotoshi,11855,M9e,Takanosho,8-7,1
227769,2023.07,15,12796,M17w,Hakuoho,11-4,0,uwatenage,12451,S1e,Hoshoryu,12-3,1


For each wrestler's ID, this tallies up their total number of wins.

In [49]:
win_count = {}

for r in rikishi_id_list:
    win_count[r] = len(sumo_only_wins.loc[results.rikishi1_id == r])
    
# I now create a dataframe that sorts the win tallies from highest to lowest.
sorted_win_count = sorted(win_count.items(), key=lambda x:x[1], reverse=True)

sorted_win_count_df = pd.DataFrame.from_dict(sorted_win_count)
sorted_win_count_df.columns = ['id', 'wins']
sorted_win_count_df.head()

,id,wins
0,1123,1121
1,7,956
2,89,831
3,41,827
4,39,781


I now do the same thing but for each wrestler's total number of losses.

In [50]:
loss_count = {}

for r in rikishi_id_list:
    loss_count[r] = len(sumo_only_losses.loc[results.rikishi1_id == r])
    
sorted_loss_count = sorted(loss_count.items(), key=lambda x:x[1], reverse=True)

sorted_loss_count_df = pd.DataFrame.from_dict(sorted_loss_count)
sorted_loss_count_df.columns = ['id', 'losses']
sorted_loss_count_df.head()

,id,losses
0,41,888
1,89,869
2,33,855
3,1284,744
4,13,731


Combining the dataframe for wins the with dataframe for losses.

In [51]:
won_loss_df = pd.merge(sorted_win_count_df, sorted_loss_count_df)
won_loss_df

,id,wins,losses
0,1123,1121,212
1,7,956,642
2,89,831,869
3,41,827,888
4,39,781,692
...,...,...,...
592,11965,0,1
593,12141,0,1
594,12369,0,1
595,11949,0,2


Adding wins and losses to get the total number of matches for each wrestler.

In [52]:
matches_list = []

for i in range(len(won_loss_df)):
    matches = int(
        won_loss_df[won_loss_df.id == won_loss_df["id"][i]].wins.iloc[0]
    ) + int((won_loss_df[won_loss_df.id == won_loss_df["id"][i]].losses.iloc[0]))
    matches_list.append(matches)

won_loss_df["matches"] = matches_list
won_loss_df = won_loss_df.iloc[:, [0, 3, 1, 2]]
won_loss_df

,id,matches,wins,losses
0,1123,1333,1121,212
1,7,1598,956,642
2,89,1700,831,869
3,41,1715,827,888
4,39,1473,781,692
...,...,...,...,...
592,11965,1,0,1
593,12141,1,0,1
594,12369,1,0,1
595,11949,2,0,2


With the won-loss totals in a single dataframe, now I calculate each wrestler's winning percentage.

In [53]:
pct_list = []

for i in range(len(won_loss_df)):
    pct = round(int(won_loss_df[won_loss_df.id == won_loss_df['id'][i]].wins.iloc[0]) / int(
    (won_loss_df[won_loss_df.id == won_loss_df['id'][i]].wins.iloc[0])
    + int(won_loss_df[won_loss_df.id == won_loss_df['id'][i]].losses.iloc[0])
), 2)
    pct_list.append(pct)

won_loss_df["pct"] = pct_list
won_loss_df

,id,matches,wins,losses,pct
0,1123,1333,1121,212,0.84
1,7,1598,956,642,0.60
2,89,1700,831,869,0.49
3,41,1715,827,888,0.48
4,39,1473,781,692,0.53
...,...,...,...,...,...
592,11965,1,0,1,0.00
593,12141,1,0,1,0.00
594,12369,1,0,1,0.00
595,11949,2,0,2,0.00


This takes the won_loss_df and merges it with the shikona_df

In [54]:
new_won_loss_df = pd.merge(won_loss_df, shikona_df)
new_won_loss_df = new_won_loss_df.iloc[:, [0, 5, 1, 2, 3, 4, 6, 7]]
new_won_loss_df.head()

,id,rikishi,matches,wins,losses,pct,alt_shikona2,alt_shikona3
0,1123,Hakuho,1333,1121,212,0.84,None,None
1,7,Kaio,1598,956,642,0.60,Koga,None
2,89,Aminishiki,1700,831,869,0.49,None,None
3,41,Kyokutenho,1715,827,888,0.48,None,None
4,39,Wakanosato,1473,781,692,0.53,Kogawa,None


In the section that follows, I will determine the highest rank each wrestler has achieved during the timeframe of this project. To begin with, I have created the following data dictionary that explains the ranking system of the Japan Sumo Association.

In [55]:
sumo_ranks_data_dict = {
    "yokozuna": "The highest rank in the top Makuuchi division of sumo. Promotion to this rank is only bestowed upon special consideration. Wrestlers at this rank can never be demoted.",
    "ozeki": "The second-highest rank, though it used to be the highest before the yokozuna title came into use. Promotion to ozeki is also not automatic, and demotion requires a multi-step process.",
    "sekiwake": "The third-highest rank, and the top rank that a wrestler can conceivably achieve through a consistent winning record.",
    "komusubi": "The fourth-highest rank, the lowest of the san'yaku titleholder ranks.",
    "maegashira": "After the higher san'yaku ranks, the rest of the top makuuchi division features the maegashira ranks. The highest is maegashira-1, followed by maegashira-2, maegashira-3, etc.",
    "juryo": "The second division of sumo, fixed at 28 wrestlers. Wrestlers in this division are ranked from the top starting at juryo-1.",
    "makushita": "The third-highest division, with 120 wrestlers. Rankings go down from from the top starting at makushita-1.",
    "sandanme": "The fourth-highest division. Wrestlers in this division are ranked from the top starting at sandanme-1.",
    "jonidan": "The fifth-highest division of sumo, with wrestlers ranked from the top starting at jonidan-1.",
    "jonokuchi": "The lowest division, the entry point for all wrestlers except for amateurs with  dispensation to enter at a higher rank. Wrestlers are ranked starting from jonokuchi-1.",
}

sumo_ranks_data_dict_df = pd.DataFrame.from_dict(sumo_ranks_data_dict, orient="index")
sumo_ranks_data_dict_df = sumo_ranks_data_dict_df.reset_index()
sumo_ranks_data_dict_df.columns = ["rank", "definition"]
sumo_ranks_data_dict_df = sumo_ranks_data_dict_df.style.set_properties(
    **{"text-align": "left"}
)
sumo_ranks_data_dict_df

,rank,definition
0,yokozuna,The highest rank in the top Makuuchi division of sumo. Promotion to this rank is only bestowed upon special consideration. Wrestlers at this rank can never be demoted.
1,ozeki,"The second-highest rank, though it used to be the highest before the yokozuna title came into use. Promotion to ozeki is also not automatic, and demotion requires a multi-step process."
2,sekiwake,"The third-highest rank, and the top rank that a wrestler can conceivably achieve through a consistent winning record."
3,komusubi,"The fourth-highest rank, the lowest of the san'yaku titleholder ranks."
4,maegashira,"After the higher san'yaku ranks, the rest of the top makuuchi division features the maegashira ranks. The highest is maegashira-1, followed by maegashira-2, maegashira-3, etc."
5,juryo,"The second division of sumo, fixed at 28 wrestlers. Wrestlers in this division are ranked from the top starting at juryo-1."
6,makushita,"The third-highest division, with 120 wrestlers. Rankings go down from from the top starting at makushita-1."
7,sandanme,The fourth-highest division. Wrestlers in this division are ranked from the top starting at sandanme-1.
8,jonidan,"The fifth-highest division of sumo, with wrestlers ranked from the top starting at jonidan-1."
9,jonokuchi,"The lowest division, the entry point for all wrestlers except for amateurs with dispensation to enter at a higher rank. Wrestlers are ranked starting from jonokuchi-1."


The information in rikishi1_rank includes a designation of "e" or "w" ("east" or "west" on the banzuke [ranking chart]), but as that is immaterial to ranking, we can strip it out. A simple excision of the final character doesn't work, so I can't do the following: results.rikishi1_rank.str[:-1] See, some rankings appear on the spreadsheet with an "HD" at the end, so I have to work with a list of characters to remove in order to clean this up.

In [60]:
char_to_remove = ["e", "w", "HD"]
cleaned_rankings = results["rikishi1_rank"]
for character in char_to_remove:
    cleaned_rankings = cleaned_rankings.str.replace(character, "")

results_cleaned_rank = results
results_cleaned_rank["rikishi1_rank"] = cleaned_rankings

results_cleaned_rank["rikishi1_rank"] = results_cleaned_rank["rikishi1_rank"].str.replace("Ms", "makushita") # temporarily lowercase so as not to affect "M" in "Maegashira"
results_cleaned_rank["rikishi1_rank"] = results_cleaned_rank["rikishi1_rank"].str.replace("J", "Juryo")
results_cleaned_rank["rikishi1_rank"] = results_cleaned_rank["rikishi1_rank"].str.replace("M", "Maegashira")
results_cleaned_rank["rikishi1_rank"] = results_cleaned_rank["rikishi1_rank"].str.replace("makushita", "Makushita") # now that "Maegashira" is done, change back to uppercase
results_cleaned_rank["rikishi1_rank"] = results_cleaned_rank["rikishi1_rank"].str.replace("Y1YO", "Yokozuna") # usually "Yokozuna" is either "Y1" or "Y2," but this is one outlier in the data
results_cleaned_rank["rikishi1_rank"] = results_cleaned_rank["rikishi1_rank"].str.replace("Y2YO", "Yokozuna")
results_cleaned_rank["rikishi1_rank"] = results_cleaned_rank["rikishi1_rank"].str.replace("Y1", "Yokozuna")
results_cleaned_rank["rikishi1_rank"] = results_cleaned_rank["rikishi1_rank"].str.replace("Y2", "Yokozuna")
results_cleaned_rank["rikishi1_rank"] = results_cleaned_rank["rikishi1_rank"].str.replace("O1", "Ozeki")
results_cleaned_rank["rikishi1_rank"] = results_cleaned_rank["rikishi1_rank"].str.replace("O2", "Ozeki")
results_cleaned_rank["rikishi1_rank"] = results_cleaned_rank["rikishi1_rank"].str.replace("O3", "Ozeki")
results_cleaned_rank["rikishi1_rank"] = results_cleaned_rank["rikishi1_rank"].str.replace("S1", "Sekiwake")
results_cleaned_rank["rikishi1_rank"] = results_cleaned_rank["rikishi1_rank"].str.replace("S2", "Sekiwake")
results_cleaned_rank["rikishi1_rank"] = results_cleaned_rank["rikishi1_rank"].str.replace("S3", "Sekiwake")
results_cleaned_rank["rikishi1_rank"] = results_cleaned_rank["rikishi1_rank"].str.replace("K1", "Komusubi")
results_cleaned_rank["rikishi1_rank"] = results_cleaned_rank["rikishi1_rank"].str.replace("K2", "Komusubi")
results_cleaned_rank["rikishi1_rank"] = results_cleaned_rank["rikishi1_rank"].str.replace("K3", "Komusubi")
results_cleaned_rank


,basho,day,rikishi1_id,rikishi1_rank,rikishi1_shikona,rikishi1_result,rikishi1_win,kimarite,rikishi2_id,rikishi2_rank,rikishi2_shikona,rikishi2_result,rikishi2_win
0,1983.01,1,4140,Juryo13,Chikubayama,0-1 (7-8),0,yorikiri,4306,Ms1e,Ofuji,1-0 (6-1),1
1,1983.01,1,4306,Makushita1,Ofuji,1-0 (6-1),1,yorikiri,4140,J13w,Chikubayama,0-1 (7-8),0
2,1983.01,1,1337,Juryo12,Tochitsukasa,1-0 (9-6),1,oshidashi,4323,J13e,Shiraiwa,0-1 (3-12),0
3,1983.01,1,4323,Juryo13,Shiraiwa,0-1 (3-12),0,oshidashi,1337,J12w,Tochitsukasa,1-0 (9-6),1
4,1983.01,1,4097,Juryo12,Tamakiyama,0-1 (8-7),0,yorikiri,4319,J11w,Harunafuji,1-0 (5-10),1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
227767,2023.07,15,11985,Sekiwake,Daieisho,9-6,0,hikiotoshi,11855,M9e,Takanosho,8-7,1
227768,2023.07,15,12451,Sekiwake,Hoshoryu,12-3,1,uwatenage,12796,M17w,Hakuoho,11-4,0
227769,2023.07,15,12796,Maegashira17,Hakuoho,11-4,0,uwatenage,12451,S1e,Hoshoryu,12-3,1
227770,2023.07,15,12094,Komusubi,Abi,6-9,1,yorikiri,12231,O1w,Kirishima,6-7-2,0


In [61]:
results

,basho,day,rikishi1_id,rikishi1_rank,rikishi1_shikona,rikishi1_result,rikishi1_win,kimarite,rikishi2_id,rikishi2_rank,rikishi2_shikona,rikishi2_result,rikishi2_win
0,1983.01,1,4140,Juryo13,Chikubayama,0-1 (7-8),0,yorikiri,4306,Ms1e,Ofuji,1-0 (6-1),1
1,1983.01,1,4306,Makushita1,Ofuji,1-0 (6-1),1,yorikiri,4140,J13w,Chikubayama,0-1 (7-8),0
2,1983.01,1,1337,Juryo12,Tochitsukasa,1-0 (9-6),1,oshidashi,4323,J13e,Shiraiwa,0-1 (3-12),0
3,1983.01,1,4323,Juryo13,Shiraiwa,0-1 (3-12),0,oshidashi,1337,J12w,Tochitsukasa,1-0 (9-6),1
4,1983.01,1,4097,Juryo12,Tamakiyama,0-1 (8-7),0,yorikiri,4319,J11w,Harunafuji,1-0 (5-10),1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
227767,2023.07,15,11985,Sekiwake,Daieisho,9-6,0,hikiotoshi,11855,M9e,Takanosho,8-7,1
227768,2023.07,15,12451,Sekiwake,Hoshoryu,12-3,1,uwatenage,12796,M17w,Hakuoho,11-4,0
227769,2023.07,15,12796,Maegashira17,Hakuoho,11-4,0,uwatenage,12451,S1e,Hoshoryu,12-3,1
227770,2023.07,15,12094,Komusubi,Abi,6-9,1,yorikiri,12231,O1w,Kirishima,6-7-2,0


Here I want to create a dictionary that lists each wrestler by ID number as a key, and the value for that key is a list of each rank that wrestler had during the top-division matches covered by the data. Obviously, each wrestler had a career in the lower ranks before reaching the top division in the first place, and that is not reflected here. However, the main goal with this is to eventually determine the best rank for every top-divsion wrestler, so the focus will eventually be on the higher ranks anyway.

In [64]:
rank_dict = {}
rank_list = []

for r in rikishi_id_list:
    for i in results.loc[results.rikishi1_id == r]["rikishi1_rank"].unique(): # Repetitions of a rank won't be necessary, thus unique() is sufficient.
        rank_list.append(i)
        rank_list
    rank_dict[r] = rank_list
    rank_list = []
rank_dict

{4140: ['Juryo13',
  'Makushita3',
  'Makushita2',
  'Juryo9',
  'Juryo4',
  'Juryo2',
  'Juryo10',
  'Juryo8',
  'Juryo3',
  'Juryo12',
  'Juryo1',
  'Maegashira13',
  'Maegashira14',
  'Juryo5',
  'Makushita1'],
 4306: ['Makushita1', 'Juryo12', 'Juryo8', 'Juryo11', 'Juryo7'],
 1337: ['Juryo12',
  'Juryo9',
  'Juryo5',
  'Juryo1',
  'Maegashira13',
  'Juryo2',
  'Maegashira14',
  'Maegashira8',
  'Maegashira9',
  'Maegashira1',
  'Juryo4',
  'Juryo3',
  'Maegashira12',
  'Maegashira5',
  'Komusubi',
  'Maegashira3',
  'Maegashira4',
  'Maegashira6',
  'Maegashira11',
  'Sekiwake',
  'Maegashira7',
  'Maegashira2',
  'Maegashira10',
  'Juryo6'],
 4323: ['Juryo13', 'Juryo10', 'Juryo8', 'Juryo7', 'Juryo9', 'Makushita1'],
 4097: ['Juryo12', 'Juryo11', 'Juryo13', 'Makushita7'],
 4319: ['Juryo11',
  'Juryo12',
  'Makushita3',
  'Juryo13',
  'Juryo9',
  'Juryo6',
  'Juryo2',
  'Juryo7',
  'Makushita6',
  'Makushita8'],
 4109: ['Juryo10', 'Juryo13', 'Makushita6'],
 4129: ['Juryo11'],
 4111: [

This next section trims down each list of ranks that a wrestler has had, keeping only the highest rank. So the value of the dictionary key, which includes a list of multiple ranks, is now converted into a list with a single entry, that of the highest rank achieved. I originally wanted this loop to get rid of the list completely, but that caused problems with exact matching between an element in a list using "in value" and non-exact matching using the same "in value" when the item had been converted into a string. The only way I found to prevent this kind of erroneous overriding was to keep the adjusted value as a list.

In [63]:
rank_dict_temp = {}

for key, value in rank_dict.items():
    for rikishi_id in rank_dict.values():
        if "Yokozuna" in value:
            value = ["Yokozuna"]
            best_rank = "Yokozuna"
        elif "Ozeki" in value:
            value = ["Ozeki"]
            best_rank = "Ozeki"
        elif "Sekiwake" in value:
            value = ["Sekiwake"]
            best_rank = "Sekiwake"
        elif "Komusubi" in value:
            value = ["Komusubi"]
            best_rank = "Komusubi"
        elif "Maegashira1" in value:
            value = ["Maegashira1"]
            best_rank = "Maegashira1"
        elif "Maegashira2" in value:
            value = ["Maegashira2"]
            best_rank = "Maegashira2"
        elif "Maegashira3" in value:
            value = ["Maegashira3"]
            best_rank = "Maegashira3"
        elif "Maegashira4" in value:
            value = ["Maegashira4"]
            best_rank = "Maegashira4"
        elif "Maegashira5" in value:
            value = ["Maegashira5"]
            best_rank = "Maegashira5"
        elif "Maegashira6" in value:
            value = ["Maegashira6"]
            best_rank = "Maegashira6"
        elif "Maegashira7" in value:
            value = ["Maegashira7"]
            best_rank = "Maegashira7"
        elif "Maegashira8" in value:
            value = ["Maegashira8"]
            best_rank = "Maegashira8"
        elif "Maegashira9" in value:
            value = ["Maegashira9"]
            best_rank = "Maegashira9"
        elif "Maegashira10" in value:
            value = ["Maegashira10"]
            best_rank = "Maegashira10"
        elif "Maegashira11" in value:
            value = ["Maegashira11"]
            best_rank = "Maegashira11"
        elif "Maegashira12" in value:
            value = ["Maegashira12"]
            best_rank = "Maegashira12"
        elif "Maegashira13" in value:
            value = ["Maegashira13"]
            best_rank = "Maegashira13"
        elif "Maegashira14" in value:
            value = ["Maegashira14"]
            best_rank = "Maegashira14"
        elif "Maegashira15" in value:
            value = ["Maegashira15"]
        elif "Maegashira16" in value:
            value = ["Maegashira16"]
        elif "Maegashira17" in value:
            value = ["Maegashira17"]
        elif "Maegashira18" in value:
            value = ["Maegashira18"]
        elif "Maegashira19" in value:
            value = ["Maegashira19"]
        elif "Maegashira20" in value:
            value = ["Maegashira20"]
        elif "Maegashira21" in value:
            value = ["Maegashira21"]
        elif "Maegashira22" in value:
            value = ["Maegashira22"]
        elif "Maegashira23" in value:
            value = ["Maegashira23"]
        elif "Maegashira24" in value:
            value = ["Maegashira24"]
        elif "Maegashira25" in value:
            value = ["Maegashira25"]
        elif "Maegashira26" in value:
            value = ["Maegashira26"]
        elif "Maegashira27" in value:
            value = ["Maegashira27"]
        elif "Maegashira28" in value:
            value = ["Maegashira28"]
        elif "Maegashira29" in value:
            value = ["Maegashira29"]
        elif "Maegashira30" in value:
            value = ["Maegashira30"]
        elif "Juryo1" in value:
            value = ["Juryo1"]
        elif "Juryo2" in value:
            value = ["Juryo2"]
        elif "Juryo3" in value:
            value = ["Juryo3"]
        elif "Juryo4" in value:
            value = ["Juryo4"]
        elif "Juryo5" in value:
            value = ["Juryo5"]
        elif "Juryo6" in value:
            value = ["Juryo6"]
        elif "Juryo7" in value:
            value = ["Juryo7"]
        elif "Juryo8" in value:
            value = ["Juryo8"]
        elif "Juryo9" in value:
            value = ["Juryo9"]
        elif "Juryo10" in value:
            value = ["Juryo10"]
        elif "Juryo11" in value:
            value = ["Juryo11"]
        elif "Juryo12" in value:
            value = ["Juryo12"]
        elif "Juryo13" in value:
            value = ["Juryo13"]
        elif "Juryo14" in value:
            value = ["Juryo14"]
        elif "Juryo15" in value:
            value = ["Juryo15"]
        elif "Juryo16" in value:
            value = ["Juryo16"]
        elif "Juryo17" in value:
            value = ["Juryo17"]
        elif "Juryo18" in value:
            value = ["Juryo18"]
        elif "Juryo19" in value:
            value = ["Juryo19"]
        elif "Juryo20" in value:
            value = ["Juryo20"]
        elif "Juryo21" in value:
            value = ["Juryo21"]
        elif "Juryo22" in value:
            value = ["Juryo22"]
        elif "Juryo23" in value:
            value = ["Juryo23"]
        elif "Juryo24" in value:
            value = ["Juryo24"]
        elif "Juryo25" in value:
            value = ["Juryo25"]
        elif "Juryo26" in value:
            value = ["Juryo26"]
        elif "Juryo27" in value:
            value = ["Juryo27"]
        elif "Juryo28" in value:
            value = ["Juryo28"]
        elif "Juryo29" in value:
            value = ["Juryo29"]
        elif "Juryo30" in value:
            value = ["Juryo30"]
        elif "Makushita1" in value:
            value = ["Makushita1"]
        elif "Makushita2" in value:
            value = ["Makushita2"]
        elif "Makushita3" in value:
            value = ["Makushita3"]
        elif "Makushita4" in value:
            value = ["Makushita4"]
        elif "Makushita5" in value:
            value = ["Makushita5"]
        elif "Makushita6" in value:
            value = ["Makushita6"]
        elif "Makushita7" in value:
            value = ["Makushita7"]
        elif "Makushita8" in value:
            value = ["Makushita8"]
        elif "Makushita9" in value:
            value = ["Makushita9"]
        elif "Makushita10" in value:
            value = ["Makushita10"]
        elif "Makushita11" in value:
            value = ["Makushita11"]
        elif "Makushita12" in value:
            value = ["Makushita12"]
        elif "Makushita13" in value:
            value = ["Makushita13"]
        elif "Makushita14" in value:
            value = ["Makushita14"]
        elif "Makushita15" in value:
            value = ["Makushita15"]
        elif "Makushita16" in value:
            value = ["Makushita16"]
        elif "Makushita17" in value:
            value = ["Makushita17"]
        elif "Makushita18" in value:
            value = ["Makushita18"]
        elif "Makushita19" in value:
            value = ["Makushita19"]
        elif "Makushita20" in value:
            value = ["Makushita20"]
        elif "Makushita21" in value:
            value = ["Makushita21"]
        elif "Makushita22" in value:
            value = ["Makushita22"]
        elif "Makushita23" in value:
            value = ["Makushita23"]
        elif "Makushita24" in value:
            value = ["Makushita24"]
        elif "Makushita25" in value:
            value = ["Makushita25"]
        elif "Makushita26" in value:
            value = ["Makushita26"]
        elif "Makushita27" in value:
            value = ["Makushita27"]
        elif "Makushita28" in value:
            value = ["Makushita28"]
        elif "Makushita29" in value:
            value = ["Makushita29"]
        elif "Makushita30" in value:
            value = ["Makushita30"]
    rank_dict_temp[key] = value

# In what follows, I create a dictionary that features a wrestler's ID as key and top rank as value, this value cleaned up to be a string instead of a list.
# However, in the next cell, I figured out a way to work with rank_dict_temp, so this new dictionary is not neccessary. Still, I'm leaving the code here
# in case I need to reference it in the future.

# top_rank_dict = {}
# for key, value in rank_dict_temp.items():
#     value_to_string = value[0]
#     top_rank_dict[key] = value_to_string

# This creates a dataframe with each wrestler's ID as well as the top rank achieved by that wrestler.
top_rank_df = pd.DataFrame.from_dict(rank_dict_temp, orient="index")
top_rank_df = top_rank_df.reset_index()
# top_rank_df.columns = ["id", "top_rank"]
top_rank_df

,index,0
0,4140,Maegashira13
1,4306,Juryo7
2,1337,Sekiwake
3,4323,Juryo7
4,4097,Juryo11
...,...,...
592,12688,Maegashira13
593,12796,Maegashira17
594,12516,Juryo2
595,12013,Juryo3


Next step is to merge the dataframe with the top rank to the main won-loss dataframe.

In [ ]:
new_won_loss_df = pd.merge(new_won_loss_df, top_rank_df)
new_won_loss_df = new_won_loss_df.iloc[:, [0, 1, 2, 3, 4, 5, 8, 6, 7]]
new_won_loss_df

I now try to determine each wrestler's most frequently used kimarite (finishing move in a victorious match). I approach this similarly to how I determined the top rank, starting with creating a dictionary in which each wrestler's ID is the key and all that wrestler's kimarite exist as a list in the key's value. The sumo_only_wins dataframe is used to get this information, since naturally I only want to look at the kimarite that is used when the wrestler actually wins, as opposed to the results dataframe that includes both wins and losses.

In [ ]:
kimarite_dict = {}
kimarite_list = []

for r in rikishi_id_list:
    for i in sumo_only_wins[sumo_only_wins.rikishi1_id == r]["kimarite"]: # The sumo_only_wins dataframe is essential!
        kimarite_list.append(i)
    if kimarite_list == []:
        kimarite_list.append('none') # Wrestlers who have never won a match will have an empty list, so "none" is added here. 
        # kimarite_dict[8901] is an example of a wrestler with no wins and therefore no kimarite.
    kimarite_dict[r] = kimarite_list
    kimarite_list = []

Now that we have a dictionary will all kimarite for every wrestler, this next step determines each wrestler's most frequently used kimarite. This information is added to favored_kimarite_dict and then coverted into a dataframe.

In [ ]:
def most_frequent(list):
    return max(set(list), key=list.count)

favored_kimarite_dict = {}
for r in rikishi_id_list:
    favored_kimarite_dict[r] = most_frequent(kimarite_dict[r])

favored_kimarite_df = pd.DataFrame(favored_kimarite_dict.items(), columns=["id", "favored_kimarite"])

# Then this new dataframe is merged with our main dataframe.
results_cleaned = pd.merge(new_won_loss_df, favored_kimarite_df)
results_cleaned = results_cleaned.iloc[:, [0, 1, 2, 3, 4, 5, 6, 9, 7, 8]]

results_cleaned

That completes the cleanup of the first imported CSV file. Now I will move on the the next file, "banzuke.csv," which features a line for every wrestler in every tournament during the covered time period. What this file brings to the table is biographical data, some of which is constant over a wrestler's career (such as birth date), and some of which changes from tournament to tournament (such as weight). My plan is to clean this up into a new dataframe and merge it with the dataframe I just finished preparing.

In [ ]:
banzuke = pd.read_csv("data/banzuke.csv", dtype={'basho': 'string', 'id': 'int64', 'rank': 'string', 'rikishi': 'string', 'heya': 'string', 'susshin': 'string', 'birth_date': 'string', 'height': 'float64', 'weight': 'float64', 'birth_date': 'string', 'prev': 'string', 'prev_w': 'float64', 'prev_l': 'float64'})
banzuke = banzuke.drop (['prev', 'prev_w', 'prev_l'], axis=1) # I don't need these columns for the final analysis.
banzuke = banzuke.drop (['rank', 'rikishi'], axis=1) # These are important columns, but they are shared with results.csv and are being cleaned up with that file. 
banzuke

I already worked on the rank and rikishi/shikona (wrestler/ring name) information in the first dataframe, so I will now start the cleanup of banzuke.csv by addressing the shusshin (home country/prefecture) column. These steps are similar to the method wherein I tied ID numbers to shikona earlier.

In [ ]:
rikishi_shusshin_list = []

for i in banzuke["shusshin"].unique():
    rikishi_shusshin_list.append(i)

# Next to make a dictionary that holds each wrestler's unique ID number as a key, with values being each shusshin associated with that wrestler.
# The vast majority of ID numbers will only feature one shusshin, but this will make sure to include any additional shusshin entries as well.

rikishi_shusshin_dict = {}

# The following code initializes a key for every number banzuke["id"] but still only adds one shusshin as a value per key, overwriting old shusshin.
# Earlier, when I was trying to discover all the shikona (ring names) for each wrestler, it was important to preserve all the older names. That is different from what
# I want to accomplish with the shusshin entries. Most wrestlers don't change their shusshin, but a few of them do, ostensibly from moving from one prefecture in
# Japan to another. (Wrestlers from abroad still seem to keep their original countries as shusshin, even when becoming Japanese citizens.)
# Unlike with the shikona, I'm not going to completely fill out this dictionary will all shusshin for any given wrestler. Just a wrestler's current shusshin will
# be sufficient for analysis purposes.

for i in range(len(banzuke)):
    rikishi_shusshin_dict.update({banzuke["id"][i]: [banzuke["shusshin"][i]]})

# Now I turn this into a new dataframe.
shusshin_df = pd.DataFrame.from_dict(
    [([key] + value) for key, value in rikishi_shusshin_dict.items()]
)
shusshin_df.rename(columns={0: "id", 1: "shusshin"}, inplace=True)

shusshin_df.head()

I now do the same thing with heya (sumo stable) as I did with shusshin.

In [ ]:
rikishi_heya_list = []

for i in banzuke["heya"].unique():
    rikishi_heya_list.append(i)

# Next to make a dictionary that holds each wrestler's unique ID number as a key, with values being each heya associated with that wrestler.
# The vast majority of ID numbers will only feature one heya, but this will make sure to include any additional heya entries as well.

rikishi_heya_dict = {}

for i in range(len(banzuke)):
    rikishi_heya_dict.update({banzuke["id"][i]: [banzuke["heya"][i]]})  # This overwrites old entries, so only the most current heya information is retained.

# If I wanted to show all the different heya each wrestler had ever belonged to, I would add the following deactivated code.
# But, as with shusshin,this analysis is only concerned with the most recent heya to which each wrestler has belonged, which ends up being the current heya,
# or, in the case of retired wrestlers, the heya in which they were most recently involved.

# for index, row in banzuke.iterrows():
#     if row["heya"] not in rikishi_heya_dict[row["id"]]:
#         rikishi_heya_dict[row["id"]].append(row["heya"])

heya_df = pd.DataFrame.from_dict([([key] + value) for key, value in rikishi_heya_dict.items()])
heya_df.rename(columns={0: "id", 1: "heya"}, inplace=True)
heya_df.head()

Next to make a dataframe for height. As with heya, I am only concerned with the most recent entry for height, which is why I am not populating the dictionary with all values associated with each wrestler's height. I imagine most wrestlers in the top division, having been wrestling for years, are adults with heights that do not fluctuate much, if at all.

In [ ]:
rikishi_height_list = []

for i in banzuke["height"].unique():
    rikishi_height_list.append(i)

rikishi_height_dict = {}

for i in range(len(banzuke)):
    rikishi_height_dict.update({banzuke["id"][i]: [banzuke["height"][i]]})

# Since there are some wrestlers who do not have a height listed, I want to convert the "not a number" entries to zero. Using height_dict_temp, I do this below.

height_dict_temp = {}
for key, value in rikishi_height_dict.items():
    if math.isnan(value[0]):
        value = [0]
    height_dict_temp[key] = value
rikishi_height_dict = height_dict_temp # Merging the temporary dictionary back with the main rikishi_height_dict, now with 0 instead of "not a number."

# Putting this into a dataframe.
height_df = pd.DataFrame.from_dict([([key] + value) for key, value in rikishi_height_dict.items()])
height_df.rename(columns={0: "id", 1: "height"}, inplace=True)
height_df.head()

I use the same approach to make a dataframe with each wrestler's weight. Unlike with some of the other columns, I want to make sure to capture each different weight a wrestler has been listed at across various tournaments. After filling out a dictionary with all the weights for a wrestler, my aim is to end up with an "average weight."

In [ ]:
rikishi_weight_list = []

for i in banzuke["weight"]: # because I want an average of all weights each wrester has been listed at during these tournaments, I do not want to use .unique() here.
    rikishi_weight_list.append(i)

rikishi_weight_dict = {}

for i in range(len(banzuke)):
    rikishi_weight_dict.update({banzuke["id"][i]: [banzuke["weight"][i]]})

for index, row in banzuke.iterrows():
    if row["weight"] not in rikishi_weight_dict[row["id"]]:
        rikishi_weight_dict[row["id"]].append(row["weight"])

# Now that I have a dictionary listing each wrestler's ID with the differing weights that wrestler had during tournaments, now I want to create a dictionary
# that features an average of all those weights.

def average_weight(weight_list):
    return round(sum(weight_list) / len(weight_list), 1)

average_weight_dict = {}

for key, value in rikishi_weight_dict.items():
    if average_weight(value) > 0: 
        value = [average_weight(value)]
    else:
        value = [0] # There are some wrestlers who don't have an average weight because they have no entries in the "weight" column. I will mark null entries as "0".
    average_weight_dict[key] = value

# Now I turn this average weight into a new dataframe.
weight_df = pd.DataFrame.from_dict([([key] + value) for key, value in average_weight_dict.items()])
weight_df.rename(columns={0: "id", 1: "avg_weight"}, inplace=True)
weight_df.head()

For the sake of completeness, I am going to clean up the birth_date column. I assume each wrestler has only a single entry for birth_date, but theoretically I can see a situation in which a wrestler is listed with an incorrect birth_date that was later revised. So to prevent any possible errors, I am cleaning things up here.

In [ ]:
rikishi_birth_date_list = []

for i in banzuke["birth_date"].unique():
    rikishi_birth_date_list.append(i)

rikishi_birth_date_dict = {}

for i in range(len(banzuke)):
    rikishi_birth_date_dict.update({banzuke["id"][i]: [banzuke["birth_date"][i]]})
rikishi_birth_date_dict
birth_date_df = pd.DataFrame.from_dict([([key] + value) for key, value in rikishi_birth_date_dict.items()])
birth_date_df.rename(columns={0: "id", 1: "birth_date"}, inplace=True)
# ID 7806 has no listed birth_date birth_date_df.loc[birth_date_df['id']==7806]

birth_date_df.head()

As I now have a dataframes for shusshin, heya, height, weight, and birth-date, I can drop them from the dataframe made from the original "banzuke" CSV file. With this new "banzuke_cleaned" dataframe, I can then merge each of the new dataframes – for shusshin, heya, height, and weight – together. The number of rows in "banzuke_cleaned" matches the number of rows in the original "banzuke" CSV file, which means that no rows have been accidentally left off during the cleanup process. 

In [ ]:
banzuke_cleaned = banzuke.drop(["shusshin", "heya", "height", "weight", "birth_date"], axis=1)
banzuke_cleaned = pd.merge(banzuke_cleaned, shusshin_df)
banzuke_cleaned = pd.merge(banzuke_cleaned, heya_df)
banzuke_cleaned = pd.merge(banzuke_cleaned, height_df)
banzuke_cleaned = pd.merge(banzuke_cleaned, weight_df)
banzuke_cleaned = pd.merge(banzuke_cleaned, birth_date_df)
banzuke_cleaned

Knowing a wrestler's height and weight allows for calculation of body mass index (BMI). This takes each wrestler's current height along with an average of the wrestler's weight, resulting in an average BMI as opposed to a BMI at any one particular moment.

In [ ]:
bmi_list = []

for i in range(len(banzuke_cleaned)):
    if (banzuke_cleaned.loc[i, "height"] == 0):
        bmi = 0
    else:
        bmi = round(banzuke_cleaned.loc[i, "avg_weight"] / ((banzuke_cleaned.loc[i, "height"] / 100) ** 2), 1)
    bmi_list.append(bmi)
bmi_list

banzuke_cleaned["bmi"] = bmi_list
# Finished calculating BMI
banzuke_cleaned

Having extracted all the data I needed from results.csv and banzuke.csv into two dataframes, I will now perform a SQL join to combine things into a single dataframe for further examination. I've already done a lot of merging of dataframes with pandas, but I like the ease of arranging columns in SQL for the final polished dataframe as opposed to the iloc method used earlier.

In [ ]:
connection = connect(":memory:")
results_cleaned.to_sql("results_cleaned", connection)
banzuke_cleaned.to_sql("banzuke_cleaned", connection)

def sql(a_string):
    return pd.read_sql(a_string, connection)

combined_sql_query = pd.read_sql_query(
    """SELECT results_cleaned.id, results_cleaned.rikishi, results_cleaned.matches, results_cleaned.wins, results_cleaned.losses, results_cleaned.pct,
    results_cleaned.top_rank, results_cleaned.favored_kimarite, banzuke_cleaned.heya, banzuke_cleaned.shusshin, banzuke_cleaned.height,
    banzuke_cleaned.avg_weight, banzuke_cleaned.bmi, banzuke_cleaned.birth_date, results_cleaned.alt_shikona2, results_cleaned.alt_shikona3
    FROM banzuke_cleaned 
    JOIN results_cleaned 
    ON banzuke_cleaned.id = results_cleaned.id""",
    connection,
)

combined_sql_query

analysis_df = pd.DataFrame(combined_sql_query).drop_duplicates() # banzuke_cleaned still had duplicate rows for each separate tournament, so I drop them here.
analysis_df = analysis_df.sort_values(by='wins', ascending=False).reset_index(drop=True) # Wins seems the best column by which to sort.
analysis_df.to_csv("data/analysis.csv", index=False)
analysis_df

Every wrestler with 500+ wins in the top division.

In [ ]:
with_500_wins = analysis_df.loc[(analysis_df.wins >=500) & (analysis_df.pct >=.5)]
with_500_wins = with_500_wins.sort_values(by='pct', ascending=False).reset_index(drop=True)
with_500_wins


In [ ]:
plt.plot(with_500_wins["pct"], with_500_wins["top_rank"])

In [ ]:
# This doesn't yet have all possible kimarite!
hakuho_kimarite = [
    kimarite_dict[1123].count("yorikiri"),
    kimarite_dict[1123].count("oshidashi"),
    kimarite_dict[1123].count("uwatedashinage"),
    kimarite_dict[1123].count("yoritaoshi"),
    kimarite_dict[1123].count("kotenage"),
    kimarite_dict[1123].count("kainahineri"),
    kimarite_dict[1123].count("uwatenage"),
    kimarite_dict[1123].count("oshitaoshi")
]
hakuho_kimarite_labels = ["yorikiri", "oshidashi", "uwatedashinage", "yoritaoshi", "kotenage", "kainahineri", "uwatenage", "oshitaoshi"]
plt.pie(hakuho_kimarite, labels = hakuho_kimarite_labels)
plt.show()

In [ ]:
#TO DO LIST###############################################

# Ranking Data Dictionary
# Kimarite Data Dictionary

I have already added each wrestler's favored kimarite (finishing move) to the main dataframe, but below I aim to give an overview of all the kimarite that all wrestlers have used in the top division. As you can see, the techniques yorikiri and oshidashi are overwhelming favored, and after the most frequent 12 kimarite, the numbers start to drop off very quickly until the bottom results barely see any usage at all.

In [ ]:
total_kimarite_list = []

for i in sumo_only_wins["kimarite"].unique(): #unique to get a total list of all the unique kimarite
    total_kimarite_list.append(i)
total_kimarite_list.sort()

total_kimarite_list_dict = {}

for i in total_kimarite_list:
    total_kimarite_list_dict[i] = len(sumo_only_wins[sumo_only_wins["kimarite"] == i])

total_kimarite_df = pd.DataFrame(total_kimarite_list_dict.items(), columns=["kimarite", "occurances"]).sort_values(by='occurances', ascending=False)
total_kimarite_df

In what follows, I put the top 12 kimarite (which all have 2700+ occurances; the 13th most frequent kimarite drops all the way down to 1955 occurances) into a dataframe. Then I made a dataframe of the remaining lower-frequency kimarite in order to get their total sum (which happens to be an even 13000). I then make that bottom-frequency total into its own line as the 13th entry in the top-frequency dataframe. Thus, the resulting dataframe shows the 12 most-frequency kimarite with a row for "other," which indicates the sum total of lower-frequency kimarite.

In [ ]:
total_kimarite_df_top = total_kimarite_df.loc[total_kimarite_df.occurances >=2700].sort_values(by='occurances', ascending=False).reset_index(drop=True)

total_kimarite_df_bottom = total_kimarite_df.loc[total_kimarite_df.occurances <2700].sort_values(by='occurances', ascending=False).reset_index(drop=True)

total_kimarite_df_bottom.count() # There are 72 of these low-frequency kimarite
sum_of_other_kimarite = sum(total_kimarite_df_bottom['occurances']) # This adds up to exactly 13000

total_kimarite_df_top.loc[len(total_kimarite_df_top.index)] = ['72 others', sum_of_other_kimarite]

# I was going to re-sort this dataframe, but I think the visual is better seeing "other" at the bottom, since "other" represents an assortment of
# lower-used kimarite instead of being a single entity with the third-most occurances

total_kimarite_df_top

Now to visualize this with a pie chart showing the kimarite frequency. You can easily see how yorikiri and oshidashi dominate the scene. The 72 low-frequency kimarite combined don't even approach those slices. The ten kimarite ranked #3-#12 in frequency all eke out smaller slices of the overall pie.

In [ ]:
total_kimarite_df_top.groupby(["kimarite"]).sum().plot(kind="pie", y="occurances")
title = plt.title("kimarite frequency")
plt.legend('', frameon=False)

<!-- TO DO: re-do all those win/loss things, but with ID instead of Shikona -->